## The ol' fasioned way

In [1]:
from selectolax.parser import HTMLParser
from httpx import get
import os

In [2]:
def get_img_tags(query="books"):
    url = f"https://unsplash.com/s/photos/{query}"
    resp = get(url)

    if resp.status_code != 200:
        raise Exception("Error getting respose")
    
    tree = HTMLParser(resp.text)
    imgs = tree.css("figure a img")

    return imgs

In [3]:
def img_filter(url, keywords):
    return not any(x in url for x in keywords)

def get_hi_res(urls):
    # remove all the premium and profile links
    urls = [i for i in urls if img_filter(i, ['premium', 'plus', 'profile', 'placeholder'])]

    # now get the link which has the hi res image
    for _, url in enumerate(urls):
        urls[_] = url.split(',')[-1].split(' ')[1].split('?')[0]

    return urls

In [8]:
def save_images(img_urls, dest="images", tag=""):
    for _, url in enumerate(img_urls):
        resp = get(url)
        file_name = f"{tag}_{_}"
        
        if not os.path.exists(dest):
            os.makedirs(dest)
        
        with open(f"{dest}/{file_name}.jpeg", "wb") as f:
            f.write(resp.content)

In [6]:
img_nodes = get_img_tags()
img_urls = [i.attrs["srcset"] for i in img_nodes]
relevant_urls = get_hi_res(img_urls)

save_images(relevant_urls[:2])